In [1]:
import os
import json
import math
import numpy as np
import copy
from scipy.spatial.transform import Rotation

In [2]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/jackal_training_data_10')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [3]:
pixel_size = 1.4e-3 # pixel size in mm

transforms = {}
transforms["focal_length"] = 4.74 #Focal Length in mm
transforms["pixel_width"] = 3*pixel_size
transforms["pixel_height"] = transforms["pixel_width"]
transforms["w"] = 1280  # image width
transforms["h"] = 720 # image height

transforms["fov_x"] = 2*math.atan((transforms["w"]*transforms["pixel_width"])/(2*transforms["focal_length"])) # field of view in x
transforms["fl_x"] = transforms["focal_length"]/transforms["pixel_width"] # focal length/ pixel_width
transforms["fl_y"] = transforms["focal_length"]/transforms["pixel_height"]# focal length/ pixel_height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["camera_model"] = "OPENCV" # camera model

ground_truth = {}
ground_truth = transforms.copy()

In [4]:
print(transforms["pixel_width"])
print(transforms["fov_x"])

0.0042
1.031736666064982


In [5]:
frames = []
ground_truth_frames = []

starting_pts = [[0,-3], [0, -2.7], [0,-2.4], [0.0,-2.1], [0.0, -1.8], [0, -1.5]]
z = 2.5
pitch = 0.785
spin = 50

cnt = 0

sd_translation = 0.10
sd_rotation = 10

for x, y in starting_pts:
    start_transform_matrix = np.eye(4)
    start_transform_matrix[:3, :3] = Rotation.from_rotvec(np.array([pitch, 0, 0])).as_matrix()
    start_transform_matrix[0, 3] = x
    start_transform_matrix[1, 3] = y
    start_transform_matrix[2, 3] = z

    for i in range(spin):
        angle = i * 2 * np.pi / spin
        rot = Rotation.from_rotvec(np.array([0, 0, 1]) * angle).as_matrix()
        rot_matrix = np.eye(4)
        rot_matrix[:3, :3] = rot
        transform_matrix = np.matmul(rot_matrix, start_transform_matrix)

        frame = {}
        file_path = "images/{}.png".format(cnt)

        frame["file_path"] = file_path
        frame["transform_matrix"] = transform_matrix.tolist().copy()
        frames.append(frame.copy())

        
        noise_matrix = np.eye(4)
        noise_matrix[0, 3] = np.random.normal(0, sd_translation)
        noise_matrix[1, 3] = np.random.normal(0, sd_translation)
        noise_matrix[:3, :3] = Rotation.from_rotvec(np.array([0, 0, 1]) * np.random.normal(0, sd_rotation), degrees=True).as_matrix()
        print(transform_matrix)
        transform_matrix = np.matmul(noise_matrix, transform_matrix)
        print(transform_matrix)
        
        frame["transform_matrix"] = transform_matrix.tolist().copy()
        
        ground_truth_frames.append(frame.copy())
        
        
        
        cnt += 1


transforms["frames"] = frames
ground_truth["frames"] = ground_truth_frames

[[ 1.          0.          0.          0.        ]
 [ 0.          0.70738827 -0.70682518 -3.        ]
 [ 0.          0.70682518  0.70738827  2.5       ]
 [ 0.          0.          0.          1.        ]]
[[ 0.96284706 -0.19102829  0.19087622  0.85002789]
 [ 0.27004729  0.68110672 -0.68056455 -2.97036923]
 [ 0.          0.70682518  0.70738827  2.5       ]
 [ 0.          0.          0.          1.        ]]
[[ 0.9921147  -0.08865926  0.08858869  0.3759997 ]
 [ 0.12533323  0.7018103  -0.70125165 -2.9763441 ]
 [ 0.          0.70682518  0.70738827  2.5       ]
 [ 0.          0.          0.          1.        ]]
[[ 0.93901555 -0.24325292  0.24305929  1.09060837]
 [ 0.34387469  0.66424858 -0.66371984 -2.8417836 ]
 [ 0.          0.70682518  0.70738827  2.5       ]
 [ 0.          0.          0.          1.        ]]
[[ 0.96858316 -0.17592031  0.17578027  0.74606966]
 [ 0.24868989  0.68516437 -0.68461897 -2.90574948]
 [ 0.          0.70682518  0.70738827  2.5       ]
 [ 0.          0.          

In [6]:
print(transforms["frames"])
print(ground_truth["frames"])

[{'file_path': 'images/0.png', 'transform_matrix': [[0.9999999999999999, 0.0, 0.0, 0.0], [0.0, 0.7073882691671997, -0.706825181105366, -3.0], [0.0, 0.706825181105366, 0.7073882691671997, 2.5], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/1.png', 'transform_matrix': [[0.9921147013144777, -0.08865925916018154, 0.08858868551261047, 0.37599970069291266], [0.1253332335643042, 0.7018103013781817, -0.7012516534339018, -2.9763441039434335], [0.0, 0.706825181105366, 0.7073882691671997, 2.5], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/2.png', 'transform_matrix': [[0.9685831611286311, -0.17592030884093282, 0.17578027453437153, 0.7460696614945644], [0.2486898871648548, 0.6851643658952773, -0.6846189682803526, -2.9057494833858937], [0.0, 0.706825181105366, 0.7073882691671997, 2.5], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/3.png', 'transform_matrix': [[0.9297764858882513, -0.26040699016156393, 0.2601997036206793, 1.1043736580540338], [0.36812455268467786, 0.6577129790648515, -0.657189433025

In [7]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)
with open(os.path.join(SAVE_PATH,'ground_truth.json'), 'w') as outfile:
    json.dump(ground_truth, outfile, indent=4)